In [1]:
import json
from modules.training_module import TrainingModule
from modules.model_evaluator import ModelEvaluator
from configs.base_config import TrainingModuleConfig
from configs.base_config import ModelEvaluatorConfig

In [3]:
sample_train_config = json.loads('{ "region_name": "bengaluru", "region_type": "district", "train_start_date": "4/11/20", "train_end_date": "4/17/20", "model_class": "SEIR", "model_parameters": { "incubation_period": 6, "IbyCRatio": 0.59, "RbyCRatio": 0.41 }, "search_space": { "r0": [ 0.5, 6 ], "EbyCRatio": [ 0.5, 70 ], "infectious_period": [1,10] }, "search_parameters": { "max_evals": 1000 }, "training_loss_function": { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, { "metric_name": "rmsle", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] } ], "output_filepath": "seir_train.json" }')
sample_test_config = json.loads('{ "region_name": "bengaluru", "region_type": "district", "run_day": "4/17/20", "test_start_date": "4/18/20", "test_end_date": "4/24/20", "model_class": "SEIR", "model_parameters": { "infectious_period": 9.5412, "incubation_period": 6, "IbyCRatio": 0.59, "RbyCRatio": 0.41, "r0": 2.4091, "LatentEbyCRatio": { "3/22/20": 19.692374309340906, "4/1/20": 0.5781344247805147 } }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, { "metric_name": "rmsle", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] } ], "output_filepath": "seir_evaluation_output.json" }')
sample_finaltrain_config = json.loads('{ "region_name": "bengaluru", "region_type": "district", "train_start_date": "4/18/20", "train_end_date": "4/24/20", "model_class": "SEIR", "model_parameters": { "incubation_period": 6, "IbyCRatio": 0.59, "RbyCRatio": 0.41, "r0": 2.4091 }, "search_space": { "r0": [ 0.5, 6 ], "EbyCRatio": [ 0.5, 70 ], "infectious_period": [1,10] }, "search_parameters": { "max_evals": 1000 }, "training_loss_function": { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, { "metric_name": "rmsle", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] } ], "output_filepath": "seir_train.json" }')




sample_train_config = json.loads('{ "region_name": "bengaluru", "region_type": "district", "train_start_date": "4/13/20", "train_end_date": "4/19/20", "model_class": "SEIR", "model_parameters": { "incubation_period": 5}, "search_space": { "r0": [ 0.5, 6 ], "EbyCRatio": [ 0.5, 70 ], "IbyCRatio": [ 0.8, 1 ], "infectious_period": [1,10] }, "search_parameters": { "max_evals": 1000 }, "training_loss_function": { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, { "metric_name": "rmsle", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] } ], "output_filepath": "seir_train.json" }')
sample_test_config = json.loads('{ "region_name": "bengaluru", "region_type": "district", "run_day": "4/19/20", "test_start_date": "4/20/20", "test_end_date": "4/26/20", "model_class": "SEIR", "model_parameters": { "infectious_period": 9.5412, "incubation_period": 5, "IbyCRatio": 0.9, "r0": 2.4091, "LatentEbyCRatio": { "3/22/20": 19.692374309340906, "4/1/20": 0.5781344247805147 } }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, { "metric_name": "rmsle", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] } ], "output_filepath": "seir_evaluation_output.json" }')
sample_finaltrain_config = json.loads('{ "region_name": "bengaluru", "region_type": "district", "train_start_date": "4/20/20", "train_end_date": "4/26/20", "model_class": "SEIR", "model_parameters": { "incubation_period": 5, "r0": 2.4091 }, "search_space": { "r0": [ 0.5, 6 ], "EbyCRatio": [ 0.5, 70 ], "IbyCRatio": [ 0.8, 1 ], "infectious_period": [1,10] }, "search_parameters": { "max_evals": 1000 }, "training_loss_function": { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, { "metric_name": "rmsle", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] } ], "output_filepath": "seir_train.json" }')



In [4]:
sample_train_config

{'region_name': 'bengaluru',
 'region_type': 'district',
 'train_start_date': '4/13/20',
 'train_end_date': '4/19/20',
 'model_class': 'SEIR',
 'model_parameters': {'incubation_period': 5},
 'search_space': {'r0': [0.5, 6],
  'EbyCRatio': [0.5, 70],
  'IbyCRatio': [0.8, 1],
  'infectious_period': [1, 10]},
 'search_parameters': {'max_evals': 1000},
 'training_loss_function': {'metric_name': 'mape',
  'variable_weights': [{'variable': 'confirmed', 'weight': 1}]},
 'loss_functions': [{'metric_name': 'mape',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]},
  {'metric_name': 'rmsle',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]}],
 'output_filepath': 'seir_train.json'}

In [5]:
sample_test_config

{'region_name': 'bengaluru',
 'region_type': 'district',
 'run_day': '4/19/20',
 'test_start_date': '4/20/20',
 'test_end_date': '4/26/20',
 'model_class': 'SEIR',
 'model_parameters': {'infectious_period': 9.5412,
  'incubation_period': 5,
  'IbyCRatio': 0.9,
  'r0': 2.4091,
  'LatentEbyCRatio': {'3/22/20': 19.692374309340906,
   '4/1/20': 0.5781344247805147}},
 'loss_functions': [{'metric_name': 'mape',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]},
  {'metric_name': 'rmsle',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]}],
 'output_filepath': 'seir_evaluation_output.json'}

In [6]:
sample_finaltrain_config

{'region_name': 'bengaluru',
 'region_type': 'district',
 'train_start_date': '4/20/20',
 'train_end_date': '4/26/20',
 'model_class': 'SEIR',
 'model_parameters': {'incubation_period': 5, 'r0': 2.4091},
 'search_space': {'r0': [0.5, 6],
  'EbyCRatio': [0.5, 70],
  'IbyCRatio': [0.8, 1],
  'infectious_period': [1, 10]},
 'search_parameters': {'max_evals': 1000},
 'training_loss_function': {'metric_name': 'mape',
  'variable_weights': [{'variable': 'confirmed', 'weight': 1}]},
 'loss_functions': [{'metric_name': 'mape',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]},
  {'metric_name': 'rmsle',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]}],
 'output_filepath': 'seir_train.json'}

In [33]:
regions = [('ahmadabad', 'district')]#, ('bengaluru', 'district')]#, ('Pune', 'district'), ('Karnataka', 'state'), ('Rajasthan', 'state')]

resultDict = dict()
for region in regions:
    resultDict[region[0]] = dict()
   
    train_config = sample_train_config.copy()
    train_config['region_name'] = region[0]
    train_config['region_type'] = region[1]
    train_module_config = TrainingModuleConfig.parse_obj(train_config)
    trainResults = TrainingModule.from_config(train_module_config)
    resultDict[region[0]]['Train1MAPE'] = trainResults['train_metric_results'][0]['value']
    resultDict[region[0]]['Train1RMLSE'] = trainResults['train_metric_results'][1]['value']
    resultDict[region[0]]['Train1BestParams'] = trainResults['best_params']
    print("Training Complete")
    #TODO: Might need to change IbyCRatio if the seed data is available for April 20th
    
    test_config = sample_test_config.copy()
    test_config['region_name'] = region[0]
    test_config['region_type'] = region[1]
    test_config['model_parameters'].update(trainResults['best_params'])
    test_config['model_parameters'].update(trainResults['latent_params'])
    test_module_config = ModelEvaluatorConfig.parse_obj(test_config) 
    evalResults = ModelEvaluator.from_config(test_module_config)
    resultDict[region[0]]['Test1MAPE'] = evalResults[0]['value']
    resultDict[region[0]]['Test1RMLSE'] = evalResults[1]['value']
    
    print("Evaluation Complete:", resultDict[region[0]]['Test1MAPE'], resultDict[region[0]]['Test1RMLSE'])
    finalTrain_config = sample_finaltrain_config.copy()
    
    #TODO: Might need to change IbyCRatio if the seed data is available for April 20th
    
    
    finalTrain_config['region_name'] = region[0]
    finalTrain_config['region_type'] = region[1]
    finalTrain_module_config = TrainingModuleConfig.parse_obj(finalTrain_config)
    finalResults = TrainingModule.from_config(finalTrain_module_config)
    resultDict[region[0]]['Train2MAPE'] = finalResults['train_metric_results'][0]['value']
    resultDict[region[0]]['Train2RMLSE'] = finalResults['train_metric_results'][1]['value']
    resultDict[region[0]]['Train2BestParams'] = finalResults['best_params']
    
    print(resultDict)
    #TODO: Might need to change IbyCRatio if the seed data is available for April 27th before exporting final params
    
    

t = 7.90                                                
t = 7.90                                                
t = 7.90                                                                         
t = 7.90                                                                         
t = 7.90                                                                        
t = 7.90                                                                        
t = 7.90                                                                        
t = 7.90                                                                        
t = 7.90                                                                        
t = 7.90                                                                          
t = 7.90                                                                          
t = 7.90                                                                           
t = 7.90                                                           

t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                    

t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                         

t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                          
t = 7.90                                                                          
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                           
t = 7.90                                                                      

t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                         

...

t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                         

t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                         

t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                         

t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                         

t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                                            
t = 7.90                                                         

In [ ]:
test_config

In [53]:
print(resultDict)

{'ahmadabad': {'Train1MAPE': 3.34586420257681, 'Train1RMLSE': 0.044821292361338076, 'Train1BestParams': {'EbyCRatio': 0.5119807729865739, 'infectious_period': 25.201507523738194, 'r0': 2.4828844271736283}, 'Test1MAPE': 61.060297970903385, 'Test1RMLSE': 0.5082157540633349, 'Train2MAPE': 0.6813032501497117, 'Train2RMLSE': 0.010136853604869628, 'Train2BestParams': {'EbyCRatio': 0.5035720831238576, 'infectious_period': 6.540073032076016, 'r0': 1.0204607006837725}}}


In [42]:
print(resultDict)

{'ahmadabad': {'Train1MAPE': 3.188308823432515, 'Train1RMLSE': 0.04412413723656801, 'Train1BestParams': {'EbyCRatio': 0.5058787252008581, 'infectious_period': 9.687857097074652, 'r0': 3.894759235310752}, 'Test1MAPE': 65.77787876713818, 'Test1RMLSE': 0.5368232856328796, 'Train2MAPE': 0.8947810839065686, 'Train2RMLSE': 0.014590457875785454, 'Train2BestParams': {'EbyCRatio': 0.5178310056894051, 'infectious_period': 4.143561038524135, 'r0': 1.7047298818547054}}}


In [33]:
print(resultDict)

{'ahmadabad': {'Train1MAPE': 3.4217035756483347, 'Train1RMLSE': 0.04733917388335904, 'Train1BestParams': {'EbyCRatio': 0.5133938782908118, 'infectious_period': 9.857731357561642, 'r0': 3.6490526168789046}, 'Test1MAPE': 62.16189875241549, 'Test1RMLSE': 0.5146885986451338, 'Train2MAPE': 0.7978867676564523, 'Train2RMLSE': 0.010994772721702288, 'Train2BestParams': {'EbyCRatio': 0.5955917750678179, 'infectious_period': 5.634704429279182, 'r0': 1.2883939172775227}}}


In [40]:
# {'bengaluru': {'Train1MAPE': 6.672210737767751, 'Train1RMLSE': 0.06869369809529435, 'Train1BestParams': {'EbyCRatio': 0.5089131561590668, 'infectious_period': 1.1487736034282638, 'r0': 0.5006588654817549}, 'Test1MAPE': 5.240629147114487, 'Test1RMLSE': 0.08526806789447133, 'Train2MAPE': 13.624999251004883, 'Train2RMLSE': 0.14417859189818755, 'Train2BestParams': {'EbyCRatio': 0.5149582037114391, 'infectious_period': 1.421523085558079, 'r0': 0.5059425180483115}}}


# {'ahmadabad': {'Train1MAPE': 3.4217035756483347, 'Train1RMLSE': 0.04733917388335904, 'Train1BestParams': {'EbyCRatio': 0.5133938782908118, 'infectious_period': 9.857731357561642, 'r0': 3.6490526168789046}, 'Test1MAPE': 62.16189875241549, 'Test1RMLSE': 0.5146885986451338, 'Train2MAPE': 0.7978867676564523, 'Train2RMLSE': 0.010994772721702288, 'Train2BestParams': {'EbyCRatio': 0.5955917750678179, 'infectious_period': 5.634704429279182, 'r0': 1.2883939172775227}}}

In [5]:
sample_test_config = json.loads('{ "region_name": "bengaluru", "region_type": "district", "run_day": "4/19/20", "test_start_date": "4/20/20", "test_end_date": "4/26/20", "model_class": "SEIR", "model_parameters": { "infectious_period": 9.5412, "incubation_period": 5, "IbyCRatio": 0.9, "r0": 2.4091, "LatentEbyCRatio": { "3/22/20": 19.692374309340906, "4/1/20": 0.5781344247805147 }, "LatentIbyCRatio": { "3/22/20": 19.692374309340906, "4/1/20": 0.5781344247805147 } }, "loss_functions": [ { "metric_name": "mape", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] }, { "metric_name": "rmsle", "variable_weights": [ { "variable": "confirmed", "weight": 1 } ] } ], "output_filepath": "seir_evaluation_output.json" }')




In [6]:
sample_test_config

{'region_name': 'bengaluru',
 'region_type': 'district',
 'run_day': '4/19/20',
 'test_start_date': '4/20/20',
 'test_end_date': '4/26/20',
 'model_class': 'SEIR',
 'model_parameters': {'infectious_period': 9.5412,
  'incubation_period': 5,
  'IbyCRatio': 0.9,
  'r0': 2.4091,
  'LatentEbyCRatio': {'3/22/20': 19.692374309340906,
   '4/1/20': 0.5781344247805147},
  'LatentIbyCRatio': {'3/22/20': 19.692374309340906,
   '4/1/20': 0.5781344247805147}},
 'loss_functions': [{'metric_name': 'mape',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]},
  {'metric_name': 'rmsle',
   'variable_weights': [{'variable': 'confirmed', 'weight': 1}]}],
 'output_filepath': 'seir_evaluation_output.json'}

In [7]:
regions = [('ahmadabad', 'district')]
for region in regions:
    test_config = sample_test_config.copy()
    test_config['region_name'] = region[0]
    test_config['region_type'] = region[1]
    test_module_config = ModelEvaluatorConfig.parse_obj(test_config) 
    evalResults = ModelEvaluator.from_config(test_module_config)

4/1/20
t = 7.90
